# Undoing commits and changes


## Reviewing old commits

Beforing turning back the clock, we have to find out the point in the timeline that things went wrong. Review the commit history by
```bash
git log --oneline
```

If we only want to view a particular branch, do
```bash
git log <branch_name>
```

If we want to review changes across all branches,
```bash
git log --branches=*
```

## Checking out an older revision vs an old file

To revert an older commit, run
```bash
git checkout <commit_id>
```
During the normal course of development, the **HEAD** usually points to master or some other local branch, but when you check out a previous commit, HEAD no longer points to a branch — it points directly to a commit. This is called a “**detached HEAD**” state.

The point is, your development should always take place on a branch—never on a detached HEAD. This makes sure you always have a reference to your new commits. However, if you’re just looking at an old commit, it doesn’t really matter if you’re in a detached HEAD state or not.

On the other hand, **checking out an old file** 
```bash
git checkout <commit_id> <filename>
git checkout <branch> <filename>
```
**does not move the HEAD pointer**. It remains on the same branch and same commit, avoiding a 'detached head' state. You can then commit the old version of the file in a new snapshot as you would any other changes. So, in effect, this usage of git checkout on a file, serves as a way to revert back to an old version of an individual file.

## Review an old snapshot
Suppose there is a previous commit a1e8fb5 that you just want to **take a look**.
```bash
git checkout a1e8fb5
```
We can view files, compile the code etc. When we are done, 
```bash
git checkout master
```
will help us go back to the **HEAD**. If you made any changes, this checkout will be **rejected** until you **stash your changes**.

Note that your `git checkout master` will lead you to your **local HEAD of the master branch**. If you do `git log --oneline`, you will see something like
```bash
commit_id2 (HEAD -> master) crazy change
commit_id1 (origin/master) important change
...
```

## Work on an old snapshot
On the other hand, if you want to **go back in time** and discard the commits after a1e8fb5 and commit based on a1e8fb5 going forward, you cannot do 
```bash
git checkout a1e8fb5
```
, make changes and then
```bash
git commit -a
```
This is because checking out a specific commit will put the repo in a "**detached HEAD**" state. This means you are **no longer working on any branch**. In a detached state, any new commits you make will be orphaned when you change branches back to an established branch. **Orphaned commits are up for deletion by Git's garbage collector**. The garbage collector runs on a configured interval and permanently destroys orphaned commits. To prevent orphaned commits from being garbage collected, we need to ensure we are on a branch.

From the detached HEAD state, we can execute 
```bash
git checkout -b <new_branch>
``` 
This will create a **new branch** and switch to that state. You can then happily work in the new branch. Unfortunately, if you need the previous branch, maybe it was your **master branch**, this undo strategy is not appropriate.

## Undo a public commit with `git revert`

Go back to the example of
```bash
commit_id2 (HEAD -> master) crazy change
commit_id1 (origin/master) important change
...
```

At this stage, if we run
```bash
> git revert HEAD
> git log --oneline
commit_id3 (HEAD -> master) Revert "crazy changecrazy change"
commit_id2 crazy changecrazy change
commit_id1 (origin/master) important change
...
```

Here, `git revert HEAD` submits **another commit** which is the negate of the previous commit. The previous commit **still exists** in the history of the repo though. This is particularly important in cases that the commit being reverted has been pushed to remote. Being additional commit, `git push` will run smoothly. But it will throw an error if we do the following.

## Undo a local commit with `git reset --hard`

Still the same example.
```bash
commit_id2 (HEAD -> master) crazy change
commit_id1 (origin/master) important change
...
```
If we run
```bash
> git reset --hard commit_id1
> git log --oneline
commit_id1 (HEAD -> master, origin/master) important change
...
```
This method of undoing changes has the **cleanest effect on history**. Doing a reset is great for local changes however it adds complications when working with a shared remote repository as mentioned at the end of the last section. The reset local branch is considered **not up-to-date** so that cannot be pushed to remote.

## Modify the last commit by `git commit --amend`

Suppose you want to add something to the last commit but don't want waste another commit snapshot.  Once you have made more changes in the working directory and staged them for commit by using
```bash
git add <changed_files>
git commit --amend
```
This will have Git open the configured system editor and let you **modify the last commit message**. The new changes will be added to the amended commit.

## Undo changes to staging index by `git reset --mixed`

The `git add` command is used to add changes to the staging index. 
```bash
git reset 
```
is primarily used to undo the staging index changes. A 
```bash
git reset --mixed 
``` 
reset will **move any pending changes from the staging index back into the working directory**.

## Git Clean

Unlike other undo commands such as `git reset` and `git checkout`, `git clean` is mainly concerned with **untracked files**. It performs operations like `rm`. By default, you have to do the real _deletion_ by using `-f` or `--force`. Following is the common usage.

```bash
git clean -n
```
will list the files that going to be cleaned. And
```bash
git clean -f
```
realy does the cleaning work.

By default, `git clean` **does not clean directories**. To include directories, 
```bash
git clean -dn
git clean -df
```

By default, **ignored files** are not displayed or cleaned. To include ignored files,
```bash
git clean -xn
git clean -xf
```

Finally, if we want to do it **interactively**, replace `-f` by `-i`. For example:
```bash
git clean -di
```

## Git Revert

`git revert` is not a traditional undo operation. Instead of removing the commit from the project history, it figures out how to **invert the changes introduced by the commit** and **appends a new commit** with the resulting inverse content. This prevents Git from losing history, which is important for the integrity of your revision history and for reliable collaboration.

`git revert` expects a **commit ref** passed in and will not execute without one. If we run
```bash
git revert HEAD
```
this will revert the latest commit.

A few common options:
```bash
git revert -n
git revert --no-commit
```
Instead of a new commit, `-n` will make the inverse change into the **staging index**. 

`git revert` can take a commit id. By giving a commit id, it will figure out the difference between that commit and the HEAD.
```bash
git revert <commid_id>
```

## Git Reset

`git reset` has three primary forms of invocation: `--soft`, `--mixed` and `--hard`. The three arguments each correspond to Git's **three internal state management mechanism's**, The **Commit Tree** (**HEAD**), The **Staging Index**, and The **Working Directory**. It is sometimes called Git's "**three trees**". `--soft` only operates on the _Commit History_, `--mixed` operates on _both the Commit History and the Staged Snapshot_ and `--hard` operates on _all three tress_.

At a surface level, `git reset` is similar in behavior to `git checkout`. Where `git checkout` solely operates on the **HEAD ref pointer**, `git reset` will **move** the **HEAD ref pointer** and the **current branch ref pointer**. Suppose the commit history is
```
A -> B -> C -> D (HEAD,master)
```
After
```bash
git checkout B
```
The status becomes
```
A -> B (HEAD) -> C -> D (master)
```
This is the so-called "**detached HEAD** situation.

Instead of `git checkout B`, if we run
```bash
git reset B
```
the status then becomes
```
A -> B (HEAD,master)  C -> D
```

**`--hard`** is the most direct, **DANGEROUS**, and frequently used option. It will update the **ref pointers** to the specific commit, and **reset the Staging Index and Working Direcotry** to match that specific commit as well. It means any tracked will have the same contents in three trees after a hard reset. **Any changes to Staging Index and Working Directory will be lost**. This data loss cannot be undone, this is critical to take note of.

**`--mixed`** is the **default operating mode**. The **ref pointers are updated**. The **Staging Index is reset** to the state of the specified commit. Any changes that have been undone from the Staging Index are moved to the Working Directory.

**`--soft`** **updates the ref pointers** and **the Staging Index and the Working Directory are left untouched**.

## Reset vs Revert

Whereas `git revert` is designed to safely undo a public commit, `git reset` is designed to undo local changes to the Staging Index and Working Directory. Because of their distinct goals, the two commands are implemented differently: resetting completely removes a changeset, whereas reverting maintains the original changeset and uses a new commit to apply the undo.

The `git reset` command is frequently encountered while preparing the staged snapshot, i.e. **unstaging a file**. 

The `git reset` command is frequently used when trying to **remove a local commit**.

You should **never use `git reset <commit>` when any snapshots after `<commit>` have been pushed to a public repository**. After publishing a commit, you have to assume that other developers are reliant upon it.